In [1]:
!python.exe -m pip install --upgrade pip
!pip install -q -U watermark
!pip install -q spacy
!pip install -q tensorflow
%env TF_CPP_MIN_LOG_LEVEL=3
!pip install -q transformers

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
# 1. Imports
import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall, AUC
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from keras.saving import register_keras_serializable
from tensorflow.keras.layers import Layer, Dense
from transformers import TFDistilBertModel, DistilBertConfig
from tensorflow.keras.metrics import Precision, Recall, AUC
import warnings
warnings.filterwarnings('ignore')

C:\Users\BHAGYASHREE\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
%reload_ext watermark
%watermark -a "Bhagyashree"

Author: Bhagyashree



In [4]:
# 2.a Loading Training Data
training_data = pd.read_csv('training_data.txt', header=None, delimiter=';')

# 2.b Loading Test Data
test_data = pd.read_csv('test_data.txt', header=None, delimiter=';')

In [5]:
# 3. Adjusting Column Names
training_data = training_data.rename(columns={0: 'text', 1: 'sentiment'})
test_data = test_data.rename(columns={0: 'text', 1: 'sentiment'})

In [6]:
training_data.shape

(16000, 2)

In [7]:
test_data.shape

(2000, 2)

In [8]:
training_data.head()

,text,sentiment
0,i am feeling completely overwhelmed i have two...,fear
1,i have the feeling she was amused and delighted,joy
2,i was able to help chai lifeline with your sup...,joy
3,i already feel like i fucked up though because...,anger
4,i still love my so and wish the best for him i...,sadness


In [9]:
# 7. Sentiments in Training Data
training_data['sentiment'].value_counts()


sentiment
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

## There is a Python function that applies preprocessing using SpaCy, taking the text data as input. This data is passed through the dictionary. Why? For example, if I want to change the verb form of a word, I need the dictionary to handle this transformation.

## This is exactly what we do here — passing the text through the dictionary. Then, I will extract the lemma. What is the lemma? It's essentially the root form of a word. The nlp object processes the text and breaks it down into tokens. In other words, it converts the text into smaller components (tokens).

## After this, based on the dictionary, I extract the lemma for each token, which is its root form. I then convert it to lowercase (lower), and apply strip to clean up any unnecessary spaces.

## This entire process is done inside a list comprehension, which acts as a loop in Python. For each token in my list of tokens (referred to as doc), I check if the token is valid. If so, I extract its lemma, convert it to lowercase, and apply stripto remove stop words and other unnecessary characters.

In [10]:
!python -m spacy download en_core_web_md -q

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [11]:
nlp = spacy.load('en_core_web_md')

In [12]:
# 10. Definition of the 'preprocess_text' Function, Which Takes a Text as a Parameter
def preprocess_text(text):

    # 10.a Process the text using the SpaCy model
    doc = nlp(text)

    # 10.b Create a list of lemmatized tokens, converted to lowercase, stripped of whitespace,
    # excluding stopwords
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

    # 10.c Return the processed tokens as a single string, joined with spaces
    return ' '.join(tokens)

In [13]:
training_data['processed_text'] = training_data['text'].apply(preprocess_text)
test_data['processed_text'] = test_data['text'].apply(preprocess_text)

In [14]:
training_data.head()

,text,sentiment,processed_text
0,i am feeling completely overwhelmed i have two...,fear,feel completely overwhelmed strategy help feel...
1,i have the feeling she was amused and delighted,joy,feeling amuse delight
2,i was able to help chai lifeline with your sup...,joy,able help chai lifeline support encouragement ...
3,i already feel like i fucked up though because...,anger,feel like fuck not usually eat morning
4,i still love my so and wish the best for him i...,sadness,love wish good long tolerate effect bm life fa...


## Why are we simplified using lemmatization here is the reason its when the data gets overwhelmed 
## Simplification Steps:
## 1:Lemmatization: Consider the first line, "Feeling completely overwhelmed." As humans, we know that "feeling" is the gerund form of the verb feel. However, the computer cannot distinguish between gerund and verb forms. Instead of using "feeling", we use just the root form: "feel". This eliminates the need for word forms and focuses only on their roots.
## 2:Removing Stopwords: Words like pronouns, adverbs, and common connectors (e.g., "and," "but," "the") are often unnecessary for analysis. These words are stopwords — commonly occurring words with little contextual value in most cases.
## 3:Reducing Redundancy: Removing verbs' full forms and keeping just their roots reduces unnecessary repetition, simplifies text data, and avoids creating excessively large matrices.

In [15]:
# 14. Create the Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

## Let's build the first version of our model. The first step is vectorization.

## Since the data is in text format, and we cannot perform mathematical operations on text, we need to convert it into a numerical representation.

## There are numerous strategies for this, and one of them is TF-IDF (Term Frequency-Inverse Document Frequency):

NameError: name 'train_tfidf' is not defined